# 숙제
- 네이버 기사 (끝자리 30108) 크롤링
    - https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108
- 전처리
    - 문장을 나중에 나누어야 하므로 문장 구분자 남겨둘 것
    - 아니면 문장부터 일단 구분하고 전처리
- 키워드 생성 (명사)
    - tokenized된 스트링 1개짜리에 대해서 생성
- 문장 나누기
- 문장 중요도 계산하기
- 중요도 높은순 문장 3개 출력

### 최종 결과

#### tokenizer
- Mecab
    - NNP, NNG, SN

<br>

#### keywords
min_ratio=0.01, max_ratio=0.5
```python
{'빅', '활용', '기업', '계획', '혁신', '분야', '구축', '플랫', '폼', '데이터', '센터'}
```

<br>

#### key sentences
```python
Rank 1 -> 기사 10번째 문장
Importance Score 5.0
---
raw: 빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다
tokenized: 빅 데이터 데이터 활용 혁신 장 문재 정부 경제 성장 핵심 요소
===

Rank 2 -> 기사 12번째 문장
Importance Score 4.923076923076923
---
raw: 이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다
tokenized: 맥락 속 빅 데이터 센터 공공 민간 협업 활용 양질 데이터 생산 구축 플랫 폼 수집 분석 유통 역할 담당
===

Rank 3 -> 기사 마지막 문장
Importance Score 4.545454545454546
---
raw: 유영민 과기정통부 장관은 “오늘 출범식은 대한민국이 데이터 강국으로 가기 위한 초석을 놓은 자리”라며 “세계 주요국들보다 데이터 경제로 나아가는 발걸음이 다소 늦었지만, 빅데이터 플랫폼과 센터를 지렛대로 우리나라의 낙후된 데이터 생태계를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다”고 밝혔다.
tokenized: 유영민 과기 정통부 장관 오늘 출범식 대한민국 데이터 강국 초석 자리 세계 주요 국 데이터 경제 발걸음 빅 데이터 플랫 폼 센터 지렛대 우리나라 낙후 데이터 생태계 혁신 기업 경쟁력 단계 제고 정책 역량 집중
===
```

In [66]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import re
import kss

In [67]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108'
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [68]:
news_with_tags = soup.select('div._article_body_contents')[0]
news_with_tags

<div class="_article_body_contents" id="articleBodyContents">
<!-- 본문 내용 -->
<!-- TV플레이어 -->
<!-- // TV플레이어 -->
<script type="text/javascript">
// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}
</script>
<strong class="media_end_summary">과기정통부, 22일 유영민 장관 등 참석해 기념행사<br/>2021년까지 1516억원 투입, 5100여종 데이터 구축<br/>민간 클라우드 통한 외부연계체계도.."개방성 강화"</strong>[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.<br/><br/>22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.<br/><br/><strong>◇10개 분야 100개 센터..3년간 1516억원 투입<br/><br/><table align="CENTER" bgcolor="ffffff" border="0" cellpadding="0" cellspacing="5" width="466"><tr><td bgcolor="ffffff" style="PADDING-RIGHT: 2px; PADDING-LEFT: 2px; PADDING-BOTTOM: 2px; PADDING-TOP: 2px;"><table bgcolor="ffffff" border="0" cellpadding="0" cellspacing="5"><tr><td><sp

# STEP1. 문장 추출
1. string decode -> br, strong 태그 텍스트로 split
2. 마침표+공백 한칸으로 split
3. <> 태그 제거
4. 불필요한 부분 삭제하기

## string decode -> br, strong 태그 텍스트로 split

In [69]:
sentence_split = [re.split('<[/]*strong.*>', d1)
                     for d1 in news_with_tags.decode().split('<br/>')
                     if re.split('<[/]*strong.*>', d1) != ['']]
sentence_split

[['<div class="_article_body_contents" id="articleBodyContents">\n<!-- 본문 내용 -->\n<!-- TV플레이어 -->\n<!-- // TV플레이어 -->\n<script type="text/javascript">\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n</script>\n',
  '과기정통부, 22일 유영민 장관 등 참석해 기념행사'],
 ['2021년까지 1516억원 투입, 5100여종 데이터 구축'],
 ['민간 클라우드 통한 외부연계체계도.."개방성 강화"',
  '[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.'],
 ['22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.'],
 ['', '◇10개 분야 100개 센터..3년간 1516억원 투입'],
 ['<table align="CENTER" bgcolor="ffffff" border="0" cellpadding="0" cellspacing="5" width="466"><tr><td bgcolor="ffffff" style="PADDING-RIGHT: 2px; PADDING-LEFT: 2px; PADDING-BOTTOM: 2px; PADDING-TOP: 2px;"><table bgcolor="ffffff" border="0" cellpadding="0" cellspacing="5"><tr><td><span class="end_photo_org"><img

## 마침표+공백 한칸으로 split

In [70]:
sentences = []
for splist in sentence_split:
    #print(sp)
    
    for sentence in splist:
#         print(sentence)
#         print('-')
        if '. ' in sentence:
#             print(sentence.split('. '))
            sentences += sentence.split('. ')
        else:
            sentences.append(sentence)

    #print(type(sp[0]))

In [71]:
sentences

['<div class="_article_body_contents" id="articleBodyContents">\n<!-- 본문 내용 -->\n<!-- TV플레이어 -->\n<!-- // TV플레이어 -->\n<script type="text/javascript">\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n</script>\n',
 '과기정통부, 22일 유영민 장관 등 참석해 기념행사',
 '2021년까지 1516억원 투입, 5100여종 데이터 구축',
 '민간 클라우드 통한 외부연계체계도.."개방성 강화"',
 '[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다',
 '새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다',
 '10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.',
 '22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다',
 '유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.',
 '',
 '◇10개 분야 100개 센터..3년간 1516억원 투입',
 '<table align="CENTER" bgcolor="ffffff" border="0" cellpadding="0" cellspacing="5" width="466"><tr><td bgcolor="ffffff" style="PADDING-RIGHT: 2px; PADDING-LEFT: 2px; PADDING-BOTTOM: 2px; PADDING-TOP: 2px;"><table bgcolor="ffffff" border="0" cellpadding="0" cellspacing="5"><tr><td><span class="end_photo_org"><img al

## <> 태그 제거

In [72]:
sentences = [re.sub('<.+>', ' ', s) for s in sentences
                if re.sub('<.+>', ' ', s) not in ['', ' ', '  ']]
sentences

[' \n \n \n \n \n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n \n',
 '과기정통부, 22일 유영민 장관 등 참석해 기념행사',
 '2021년까지 1516억원 투입, 5100여종 데이터 구축',
 '민간 클라우드 통한 외부연계체계도.."개방성 강화"',
 '[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다',
 '새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다',
 '10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.',
 '22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다',
 '유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.',
 '◇10개 분야 100개 센터..3년간 1516억원 투입',
 '빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다',
 '문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.',
 '이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다',
 '과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.',
 '대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅

### TEST: KSS (Korean Sentence Splitter)
기사 도입부 첫 파트를 제외하면 문장을 성공적으로 분류하는 것을 확인

In [73]:
kss.split_sentences(news_with_tags.text)

['// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다.',
 '새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다.',
 '10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.',
 '22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다.',
 '유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.',
 '◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다.',
 '문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.',
 '이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다.',
 '과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.',
 '대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공

## 불필요한 부분 제거하기
- '오류를 우회하기 위한~'
- 기자 이메일주소 이후 텍스트

In [74]:
email_idx = 0
email_ptn = '[a-zA-Z0-9_\-.]+@[a-z]+(?:\.[a-z]+)+'
sentences_raw = []

for sentence in sentences:
    if '오류를 우회하기 위한' not in sentence:
        if re.findall(email_ptn, sentence):
            break
        sentences_raw.append(sentence)
    else:
        pass

In [75]:
sentences_raw

['과기정통부, 22일 유영민 장관 등 참석해 기념행사',
 '2021년까지 1516억원 투입, 5100여종 데이터 구축',
 '민간 클라우드 통한 외부연계체계도.."개방성 강화"',
 '[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다',
 '새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다',
 '10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.',
 '22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다',
 '유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.',
 '◇10개 분야 100개 센터..3년간 1516억원 투입',
 '빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다',
 '문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.',
 '이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다',
 '과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.',
 '대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다',
 '이를 통해 데이터 생태계를 혁신하

#### 재사용을 위한 함수 랩핑

In [1]:
def get_text_from_naver_news(url):
    #url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108'
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    news_with_tags = soup.select('div._article_body_contents')[0]
    
    sentences = []
    for splist in sentence_split:
        for sentence in splist:
            if '. ' in sentence:
                sentences += sentence.split('. ')
            else:
                sentences.append(sentence)
                
    sentences = [re.sub('<.+>', ' ', s) for s in sentences
                if re.sub('<.+>', ' ', s) not in ['', ' ', '  ']]
    
    email_idx = 0
    email_ptn = '[a-zA-Z0-9_\-.]+@[a-z]+(?:\.[a-z]+)+'
    sentences_raw = []

    for sentence in sentences:
        if '오류를 우회하기 위한' not in sentence:
            if re.findall(email_ptn, sentence):
                break
            sentences_raw.append(sentence)
        else:
            pass
        
    return sentences_raw

# STEP 2. Tokenize - Mecab (NNG, NNP, SN)

In [76]:
from konlpy.tag import Mecab
mecab = Mecab()
mecab_docs = [mecab.pos(s, flatten=True) for s in sentences_raw]
mecab_docs
#mecab_docs_nnp_nng = [[token[0] for token in mecab_docs[i] if token[1] in ['NNG', 'NNP']] for i in range(4)]

[[('과기', 'NNG'),
  ('정통부', 'NNG'),
  (',', 'SC'),
  ('22', 'SN'),
  ('일', 'NNBC'),
  ('유영민', 'NNP'),
  ('장관', 'NNG'),
  ('등', 'NNB'),
  ('참석', 'NNG'),
  ('해', 'XSV+EC'),
  ('기념행사', 'NNG')],
 [('2021', 'SN'),
  ('년', 'NNBC'),
  ('까지', 'JX'),
  ('1516', 'SN'),
  ('억', 'NR'),
  ('원', 'NNBC'),
  ('투입', 'NNG'),
  (',', 'SC'),
  ('5100', 'SN'),
  ('여종', 'NNG'),
  ('데이터', 'NNG'),
  ('구축', 'NNG')],
 [('민간', 'NNG'),
  ('클라우드', 'NNP'),
  ('통한', 'VV+ETM'),
  ('외부', 'NNG'),
  ('연계', 'NNG'),
  ('체계', 'NNG'),
  ('도', 'JX'),
  ('.', 'SF'),
  ('."', 'SY'),
  ('개방', 'NNG'),
  ('성', 'XSN'),
  ('강화', 'NNG'),
  ('"', 'SY')],
 [('[', 'SSO'),
  ('이', 'MM'),
  ('데일리', 'NNP'),
  ('이재운', 'NNP'),
  ('기자', 'NNG'),
  (']', 'SSC'),
  ('국가', 'NNG'),
  ('차원', 'NNG'),
  ('의', 'JKG'),
  ('빅', 'NNG'),
  ('데이터', 'NNG'),
  ('활용', 'NNG'),
  ('시대', 'NNG'),
  ('가', 'JKS'),
  ('열린다', 'VV+EC')],
 [('새로운', 'VA+ETM'),
  ('산업', 'NNG'),
  ('창출', 'NNG'),
  ('과', 'JC'),
  ('기존', 'NNG'),
  ('산업', 'NNG'),
  ('의', 'JKG'),
  ('변화', 'NN

In [77]:
mecab_docs_nnp_nng = [[token[0] for token in mecab_docs[i]
                           if token[1] in ['NNG', 'NNP', 'SN']]
                           for i in range(len(mecab_docs))]

In [78]:
mecab_sentence_joined = [' '.join(doc) for doc in mecab_docs_nnp_nng]
mecab_sentence_joined

['과기 정통부 22 유영민 장관 참석 기념행사',
 '2021 1516 투입 5100 여종 데이터 구축',
 '민간 클라우드 외부 연계 체계 개방 강화',
 '데일리 이재운 기자 국가 차원 빅 데이터 활용 시대',
 '산업 창출 기존 산업 변화 혁신 장 센터 문',
 '10 분야 데이터 경제 발전 정부 청사진 현실 구현 계획',
 '22 과학 기술 정보 통신부 서울 중구 대한 상공 회의소 데이터 생태 조성 혁신 성장 기반 마련 빅 데이터 플랫 폼 센터 출범식 행사 개최',
 '유영민 과기 정통부 장관 노웅래 국회 과학 기술 정보 방송 통신 위원회 위원장 300 참가',
 '10 분야 100 센터 3 간 1516 투입',
 '빅 데이터 데이터 활용 혁신 장 문재 정부 경제 성장 핵심 요소',
 '문재인 대통령 데이터 활용 정보 보호 보안 중요 강조',
 '맥락 속 빅 데이터 센터 공공 민간 협업 활용 양질 데이터 생산 구축 플랫 폼 수집 분석 유통 역할 담당',
 '과기 정통부 분야 플랫 폼 10 연계 기관 센터 100 구축 3 1516 투입 계획 올해 640 규모 사업 추진',
 '대상 분야 금융 카드 환경 한국 수자원 공사 문화 한국 문화 정보원 교통 한국 교통 연구원 헬 스케어 국립암센터 유통 소비 매일 방송 통신 중소기업 비즈 지역 경제 경기도 청 산림 한국 임업 흥원 1 공모 72 빅 데이터 센터 선정 다음 달 8 2 공모 28 추가 선정 100 지원 운영 계획',
 '데이터 생태계 혁신 기업 경쟁력 제고 역할 수행',
 '주요 활용 전략 사례 빅 데이터 활용 신 新 시장 창출 방안',
 '금융 플랫 폼 경우 소 상공 신용 평가 고도 금융 취약 계층 대상 금리 대출 2 절감 연간 1 신규 대출 창출 전망',
 '유통 소비 중소기업 플랫 폼 소상 공인 중소기업 폐업 감소 문화 플랫 폼 문화 예술 관람 생활 체육 참여 방안 모색',
 '의료비 절감 헬스 케어 기업 매출 향상 산업 육성 통신 산림 눈길',
 '2021 5100 여종 데이터 구축 알고리즘 제

In [79]:
len(mecab_sentence_joined)

28

# STEP 3. 키워드 생성 (명사)

In [80]:
tokens = [token for sentencelist in mecab_sentence_joined
                        for token in sentencelist.split()]
print(tokens)

['과기', '정통부', '22', '유영민', '장관', '참석', '기념행사', '2021', '1516', '투입', '5100', '여종', '데이터', '구축', '민간', '클라우드', '외부', '연계', '체계', '개방', '강화', '데일리', '이재운', '기자', '국가', '차원', '빅', '데이터', '활용', '시대', '산업', '창출', '기존', '산업', '변화', '혁신', '장', '센터', '문', '10', '분야', '데이터', '경제', '발전', '정부', '청사진', '현실', '구현', '계획', '22', '과학', '기술', '정보', '통신부', '서울', '중구', '대한', '상공', '회의소', '데이터', '생태', '조성', '혁신', '성장', '기반', '마련', '빅', '데이터', '플랫', '폼', '센터', '출범식', '행사', '개최', '유영민', '과기', '정통부', '장관', '노웅래', '국회', '과학', '기술', '정보', '방송', '통신', '위원회', '위원장', '300', '참가', '10', '분야', '100', '센터', '3', '간', '1516', '투입', '빅', '데이터', '데이터', '활용', '혁신', '장', '문재', '정부', '경제', '성장', '핵심', '요소', '문재인', '대통령', '데이터', '활용', '정보', '보호', '보안', '중요', '강조', '맥락', '속', '빅', '데이터', '센터', '공공', '민간', '협업', '활용', '양질', '데이터', '생산', '구축', '플랫', '폼', '수집', '분석', '유통', '역할', '담당', '과기', '정통부', '분야', '플랫', '폼', '10', '연계', '기관', '센터', '100', '구축', '3', '1516', '투입', '계획', '올해', '640', '규모', '사업', '추진', '대상', '분야', '금융', '카드

In [81]:
len(tokens)

454

In [82]:
def get_keywords(word_list, min_ratio=0.01, max_ratio=0.5):
    count_dict = {}
    for word in word_list:
        count_dict.setdefault(word, 0)
        count_dict[word] += 1
    keywords = set()
    for word, cnt in count_dict.items():
        word_percentage = count_dict[word] / len(word_list)
        if word_percentage <= max_ratio and word_percentage >= min_ratio:
            keywords.add(word)
    return keywords

keywords = get_keywords(tokens)
len(keywords)

11

In [83]:
print(keywords)

{'플랫', '센터', '활용', '계획', '구축', '혁신', '분야', '폼', '빅', '데이터', '기업'}


# STEP4. 문장 중요도 계산

In [89]:
def get_window(sentence, keywords):
    tokens = sentence.split()
    start_idx = -1
    end_idx = 0
    #print(tokens)
    for i, tk in enumerate(tokens):
        if tk in keywords:
            if start_idx == -1:
                start_idx = i
            else:
                end_idx = i
    
    if start_idx == -1:
        return []
    if end_idx == 0:
        end_idx = start_idx
    #print(start_idx, end_idx)
    
    return tokens[start_idx:end_idx+1]

In [90]:
def compute_sentence_weight(sentence_list, keywords):    
    total_words = ' '.join(sentence_list)
    keywords = get_keywords(total_words.split())
#     print(total_words)
#     print(keywords)
    
    score_list = []

    for sentence in sentence_list:
        window_tokens = get_window(sentence, keywords)
        #print('window tokens', window_tokens)
        words_num_in_window = len(window_tokens)        
        keywords_num_in_window = len([1 for word in window_tokens
                                 if word in keywords])
        
        if (words_num_in_window + keywords_num_in_window) == 0:
            score = 0
        else:
            score = keywords_num_in_window**2 / words_num_in_window
        score_list.append(score)
    
    return [(idx,score) for (idx, score) in enumerate(score_list)]

In [92]:
sentence_weights = compute_sentence_weight(mecab_sentence_joined, keywords)
sentence_weights

[(0, 0),
 (1, 2.0),
 (2, 0),
 (3, 3.0),
 (4, 1.3333333333333333),
 (5, 1.0),
 (6, 4.083333333333333),
 (7, 0),
 (8, 1.3333333333333333),
 (9, 5.0),
 (10, 2.0),
 (11, 4.923076923076923),
 (12, 2.769230769230769),
 (13, 0.4807692307692308),
 (14, 2.25),
 (15, 2.6666666666666665),
 (16, 2.0),
 (17, 1.6),
 (18, 1.0),
 (19, 2.0),
 (20, 2.3703703703703702),
 (21, 1.0),
 (22, 1.5),
 (23, 1.3888888888888888),
 (24, 3.3684210526315788),
 (25, 0),
 (26, 0.75),
 (27, 4.545454545454546)]

# STEP5. 중요도 높은 순으로 문장 3개 출력

In [93]:
top3 = [(idx,score) for (idx,score) in sorted(sentence_weights, key=lambda x: x[1], reverse=True)[:3]]
top3

[(9, 5.0), (11, 4.923076923076923), (27, 4.545454545454546)]

In [94]:
for i, (idx, score) in enumerate(top3):
    
    if idx == 0:
        print(f'Rank {i+1} -> 기사 첫번째 문장')
    elif idx == len(sentences_raw)-1:
        print(f'Rank {i+1} -> 기사 마지막 문장')
    else:    
        print(f'Rank {i+1} -> 기사 {idx+1}번째 문장')
    
    print(f'Importance Score {score}')
    print('---')
    print('raw:', sentences_raw[idx])
    print('tokenized:', mecab_sentence_joined[idx])
    print('===')
    print('')

Rank 1 -> 기사 10번째 문장
Importance Score 5.0
---
raw: 빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다
tokenized: 빅 데이터 데이터 활용 혁신 장 문재 정부 경제 성장 핵심 요소
===

Rank 2 -> 기사 12번째 문장
Importance Score 4.923076923076923
---
raw: 이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다
tokenized: 맥락 속 빅 데이터 센터 공공 민간 협업 활용 양질 데이터 생산 구축 플랫 폼 수집 분석 유통 역할 담당
===

Rank 3 -> 기사 마지막 문장
Importance Score 4.545454545454546
---
raw: 유영민 과기정통부 장관은 “오늘 출범식은 대한민국이 데이터 강국으로 가기 위한 초석을 놓은 자리”라며 “세계 주요국들보다 데이터 경제로 나아가는 발걸음이 다소 늦었지만, 빅데이터 플랫폼과 센터를 지렛대로 우리나라의 낙후된 데이터 생태계를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다”고 밝혔다.
tokenized: 유영민 과기 정통부 장관 오늘 출범식 대한민국 데이터 강국 초석 자리 세계 주요 국 데이터 경제 발걸음 빅 데이터 플랫 폼 센터 지렛대 우리나라 낙후 데이터 생태계 혁신 기업 경쟁력 단계 제고 정책 역량 집중
===



## gensim.summarization.summarizer

In [99]:
from gensim.summarization.summarizer import summarize
summarize(' '.join(sentences_raw))

'◇2021년까지 5100여종 데이터 구축..AI 알고리즘 제공도 센터는 우선 분야별 데이터 부족 문제를 해소하기 위해 올해 말까지 시장 수요가 높은 1400여종 신규 데이터를 생산ㆍ구축하고, 사업이 완료되는 2021년까지 총 5100여종 양질의 풍부한 데이터를 생산·구축해 시장에 공급할 계획이다.\n유영민 과기정통부 장관은 “오늘 출범식은 대한민국이 데이터 강국으로 가기 위한 초석을 놓은 자리”라며 “세계 주요국들보다 데이터 경제로 나아가는 발걸음이 다소 늦었지만, 빅데이터 플랫폼과 센터를 지렛대로 우리나라의 낙후된 데이터 생태계를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다”고 밝혔다.'

In [101]:
summarize(' '.join(sentences)).split('\n')

['과기정통부, 22일 유영민 장관 등 참석해 기념행사 2021년까지 1516억원 투입, 5100여종 데이터 구축 민간 클라우드 통한 외부연계체계도.."개방성 강화" [이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.',
 '◇2021년까지 5100여종 데이터 구축..AI 알고리즘 제공도 센터는 우선 분야별 데이터 부족 문제를 해소하기 위해 올해 말까지 시장 수요가 높은 1400여종 신규 데이터를 생산ㆍ구축하고, 사업이 완료되는 2021년까지 총 5100여종 양질의 풍부한 데이터를 생산·구축해 시장에 공급할 계획이다.',
 '유영민 과기정통부 장관은 “오늘 출범식은 대한민국이 데이터 강국으로 가기 위한 초석을 놓은 자리”라며 “세계 주요국들보다 데이터 경제로 나아가는 발걸음이 다소 늦었지만, 빅데이터 플랫폼과 센터를 지렛대로 우리나라의 낙후된 데이터 생태계를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다”고 밝혔다.']